In [4]:
import ollama
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
# Global chain storage
qa_chain = None
# Step 1: Load and process the PDF
def process_pdf_and_answer(pdf_file, question):
    global qa_chain
    if pdf_file is None:
        return "❌ Please upload a PDF file."
    try:
        loader = PyPDFLoader(pdf_file.name)
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        docs = text_splitter.split_documents(documents)
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        vector_store = FAISS.from_documents(docs, embeddings)
        llm = Ollama(model="llama3.2")
        qa_chain = RetrievalQA.from_llm(llm=llm, retriever=vector_store.as_retriever())
        if not question.strip():
            return "✅ PDF processed! Now enter a question."
        return qa_chain.run(question)
    except Exception as e:
        return f"❌ Error: {str(e)}"
# Step 2: Gradio Interface (Simple UI)
demo = gr.Interface(
    fn=process_pdf_and_answer,
    inputs=[
        gr.File(label="📄 Upload a PDF"),
        gr.Textbox(lines=2, label="❓ Ask a question about the PDF")],
    outputs=gr.Textbox(label="🧠 Answer from the PDF"),
    title="LangChain + Ollama PDF QA Bot",
    description="Upload a PDF and ask questions. Powered by Ollama + llama3.2 + LangChain",
    theme="soft")

In [9]:
demo.launch()

In [ ]:
demo.close()